# Run tests

This page focuses on the ways to run tests in `unittest` library.

## Run from file

If you have files listing all your tests, you can execute them using `python3 -m unittest <path to file>`. The path can be specified in two formats:

- Sections separated by `/` slashes: system-like.  
- Sections separated by `.` dots: Python imports-like.  

---

The following cell defines the unittest file that will be run later.

In [1]:
%%writefile run_tests_files/test.py
import unittest

class SomeTest(unittest.TestCase):
    def test(self):
        print("simple test")
        self.assertEqual(10,10)

Overwriting run_tests_files/test.py


You can run this test by specifying the file path to it as a parameter to the `python3 -m unittest' command.

In [2]:
!python3 -m unittest run_tests_files/test.py

simple test
.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


Or how to import this file into a Python program.

In [3]:
!python3 -m unittest run_tests_files.test

simple test
.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


## Tests package

More complex programs require more complex tests. In this case, you need to put tests in packages (folders of modules). In such a case you can just run `python -m unittest` from the folder with tests - everything that can be interpreted as python package will be executed. **Note** Unittest will only identify modules starting with `test_` as test modules.

---

The following cells prepare the structure of the package that we'll use for the example.

In [58]:
%%bash
mkdir /tmp/run_test &> /dev/null | true
mkdir /tmp/run_test/inner &> /dev/null | true

In [59]:
%%writefile /tmp/run_test/test_first.py
from unittest import TestCase

class FirstTest(TestCase):
    def test_root(self):
        print("First test")

Writing /tmp/run_test/test_first.py


In [60]:
%%writefile /tmp/run_test/inner/test_inner.py
from unittest import TestCase

class InnerTest(TestCase):
    def test_root(self):
        print("Innter test")

Writing /tmp/run_test/inner/test_inner.py


Finally we got package with one test on the current folder and one hidden on the `innter` folder.

In [61]:
!cd /tmp/run_test && tree

.
├── inner
│   └── test_inner.py
└── test_first.py

2 directories, 2 files


By running `python3 -m unittest` from the working directory we got `FirstTest` executed, but not inner test.

In [62]:
%%bash
cd /tmp/run_test
python3 -m unittest

First test


.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


This is because `unittest` doesn't recognize `inner` as a module. You can fix this by adding `__init__.py` to the `inner`.

In [63]:
%%writefile /tmp/run_test/inner/__init__.py
pass

Writing /tmp/run_test/inner/__init__.py


After that, both tests ran as expected.

In [64]:
%%bash
cd /tmp/run_test
python3 -m unittest

Innter test


..

First test



----------------------------------------------------------------------
Ran 2 tests in 0.000s

OK


## Naming

You have to name files and classes in a special way to run them with `unittest`.

By default, if you run  `unittest` without any special modifications it will run all files that start with `test`. All classes that inherit from `unitttes.TestCase` are interpreted as test classes. In each test class, all methods starting with `test` will be executed.

---

The following cells creates testing project with different options.

- Files that starts with `test` and not.
- Classes that contains `Test` in it's name and not.
- Methods that starts with `test` and not.

So we can learn in which cases naming is important and in which cases is not.

In [14]:
!rm /tmp/testing_folder &> /dev/null & mkdir /tmp/testing_folder &> /dev/null

In [15]:
%%writefile /tmp/testing_folder/test_file.py
from unittest import TestCase

class TestClass(TestCase):
    def test_method(self):
        print("test method in test class in test file")
    
    def some_method(self):
        print("some method in test class in test file")


class SomeClass(TestCase):
    def test_method(self):
        print("test method in some class in test file")

Writing /tmp/testing_folder/test_file.py


In [16]:
%%writefile /tmp/testing_folder/some_file.py
from unittest import TestCase

class TestClass(TestCase):
    def test_method(self):
        print("test method in test class in some file")

Writing /tmp/testing_folder/some_file.py


The following cell runs `unittest` without any additional configuration in the "project" folder.

In [17]:
%%bash
cd /tmp/testing_folder
python3 -m unittest

test method in some class in test file
test method in test class in test file


..
----------------------------------------------------------------------
Ran 2 tests in 0.000s

OK


You can understand from the messages:

- File name is important - because test in `some_file.py` wasn't executed.
- Name of the method is important - because `TestClass.some_method` wasn't executed.
- Name of the `TestCase` child doesn't matter - because `SomeClass.test_method` was executed.

## Skip case

If you want some child of `unittest.TestCase` not to be executed by `unittest` you can wrap it with the `unittest.skip` decorator - `unittest` will ignore it.

---

The following cell creates test file with `unittest.TestCase` wrapped in the `unittest.skip` decorator.

In [48]:
%%writefile /tmp/skip_test.py
import unittest

@unittest.skip("some text")
class TestClass(unittest.TestCase):
    def test_method(self):
        print("test_method executed")


Overwriting /tmp/skip_test.py


The following cell runs tests.

In [49]:
%%bash
cd /tmp
python3 -m unittest skip_test.py
true

s
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK (skipped=1)


As a result, no tests are executed - but one is skipped.